# Geography of the Amazon IN 
Sale Report
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json
import geopandas as gpd
import matplotlib.pyplot as plt

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file into a Pandas DataFrame
zipcode_df = pd.read_csv("../output/cleaned_Amazon Sale Report_COPY.csv")
coordinates_df = pd.read_csv("India_States-UTs.csv")

# Display sample data
zipcode_df.head()

,Order ID,Date,Status,Status (Successful),Fulfilment,ship-service-level,Style,SKU,Category,Size,ASIN,Qty,Amount,ship-city,ship-state,ship-postal-code,promotion-ids
0,405-8078784-5731545,4/30/22,Cancelled,False,Merchant,Standard,SET389,SET389-KR-NP-S,Set,S,B09KXVBD7Z,0.0,647.62,MUMBAI,MAHARASHTRA,400081.0,NaN
1,171-9198151-1101146,4/30/22,Delivered,True,Merchant,Standard,JNE3781,JNE3781-KR-XXXL,Kurta,3XL,B09K3WFS32,1.0,406.00,BENGALURU,KARNATAKA,560085.0,Amazon PLCC Free-Financing Universal Merchant ...
2,403-9615377-8133951,4/30/22,Cancelled,False,Merchant,Standard,J0341,J0341-DR-L,Western Dress,L,B099NRCT7B,0.0,753.33,PUDUCHERRY,PUDUCHERRY,605008.0,NaN
3,406-7807733-3785945,4/30/22,Delivered,True,Merchant,Standard,JNE3405,JNE3405-KR-S,Kurta,S,B081WX4G4Q,1.0,399.00,HYDERABAD,TELANGANA,500032.0,Amazon PLCC Free-Financing Universal Merchant ...
4,407-5443024-5233168,4/30/22,Cancelled,False,Amazon,Expedited,SET200,SET200-KR-NP-A-XXXL,Set,3XL,B08L91ZZXN,0.0,NaN,HYDERABAD,TELANGANA,500008.0,IN Core Free Shipping 2015/04/08 23-48-5-108


In [3]:
zipcode_df.drop(['Order ID', 'Date', 'Fulfilment', 'ship-service-level', 'Style', 'SKU', 'Size', 'ASIN', 'promotion-ids'],axis=1, inplace=True)
zipcode_df.head()

,Status,Status (Successful),Category,Qty,Amount,ship-city,ship-state,ship-postal-code
0,Cancelled,False,Set,0.0,647.62,MUMBAI,MAHARASHTRA,400081.0
1,Delivered,True,Kurta,1.0,406.00,BENGALURU,KARNATAKA,560085.0
2,Cancelled,False,Western Dress,0.0,753.33,PUDUCHERRY,PUDUCHERRY,605008.0
3,Delivered,True,Kurta,1.0,399.00,HYDERABAD,TELANGANA,500032.0
4,Cancelled,False,Set,0.0,NaN,HYDERABAD,TELANGANA,500008.0


In [4]:
zipcode_df_cleaned = zipcode_df.dropna(subset=['ship-postal-code'])
zipcode_df_cleaned['ship-postal-code'] = zipcode_df_cleaned['ship-postal-code'].astype(int)
zipcode_df_cleaned.head()


/var/folders/_k/_h4kwb151_x1sn56n2tp25j00000gp/T/ipykernel_56275/2930825787.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zipcode_df_cleaned['ship-postal-code'] = zipcode_df_cleaned['ship-postal-code'].astype(int)


,Status,Status (Successful),Category,Qty,Amount,ship-city,ship-state,ship-postal-code
0,Cancelled,False,Set,0.0,647.62,MUMBAI,MAHARASHTRA,400081
1,Delivered,True,Kurta,1.0,406.00,BENGALURU,KARNATAKA,560085
2,Cancelled,False,Western Dress,0.0,753.33,PUDUCHERRY,PUDUCHERRY,605008
3,Delivered,True,Kurta,1.0,399.00,HYDERABAD,TELANGANA,500032
4,Cancelled,False,Set,0.0,NaN,HYDERABAD,TELANGANA,500008


In [5]:
zipcode_df_cleaned["Status"].value_counts()

Status
Delivered    26
Cancelled    13
Name: count, dtype: int64

--- 

### Bring in the API of India's zip code information
#
India has a total of 28 States and 8 Union Territories with at least 720 districts.<br>
The Indian postal departmenthas allotted a unique postal code of pin code to each<br>
district/village/town/city to ensure quick delivery of postal services.

In [6]:
# # API base URL
# url = "https://api.postalpincode.in/pincode/"
# state_data = []

# # Counters
# # record_count = 1
# # set_count = 1

# print(zipcode_df_cleaned['ship-postal-code'][0]) #test
# state_url = url + str(zipcode_df_cleaned['ship-postal-code'][0])
# state_info = requests.get(state_url).json()
# print(state_info[0]["PostOffice"][0]["State"]) #test


# # Loop through all the zip codes to pull the state
# for zipcode in zipcode_df_cleaned['ship-postal-code']:
#     state_url = url + str(zipcode)
#     try:
#         state_info = requests.get(state_url).json()
#         state_name = state_info[0]["PostOffice"][0]["State"]
#         state_data.append(state_name)
#     except: # need except for all try blocks
#         print("Zip code not found. Skipping...")

# zipcode_df_cleaned["State/UT"] = state_data
# zipcode_df_cleaned.head()

---

### Make different outputs of different shipping status.

In [7]:
#  Cancelled Status
# cancelled_sts = zipcode_df_cleaned.loc[zipcode_df_cleaned['Status'] == 'Cancelled']
# cancelled_sts.head()
# cancelled_sts.to_csv("sample_Cancelled_Status.csv", index=False)


In [8]:

#  Delivered Status
# delivered_sts = zipcode_df_cleaned.loc[zipcode_df_cleaned['Status'] == 'Delivered']
# delivered_sts.head()
# delivered_sts.to_csv("sample_Delivered_Status.csv", index=False)


In [9]:

#  Returned Status
# returned_sts = zipcode_df_cleaned.loc[zipcode_df_cleaned['Status'] == 'RTS (Return to Seller)']
# returned_sts.head()
# returned_sts.to_csv("sample_RTS_Status.csv", index=False)

---

### Merging the zipcode to State/UT information

In [6]:
add_latlng = zipcode_df_cleaned.rename(columns={'ship-state': 'State/UT'})
add_latlng.head()

,Status,Status (Successful),Category,Qty,Amount,ship-city,State/UT,ship-postal-code
0,Cancelled,False,Set,0.0,647.62,MUMBAI,MAHARASHTRA,400081
1,Delivered,True,Kurta,1.0,406.00,BENGALURU,KARNATAKA,560085
2,Cancelled,False,Western Dress,0.0,753.33,PUDUCHERRY,PUDUCHERRY,605008
3,Delivered,True,Kurta,1.0,399.00,HYDERABAD,TELANGANA,500032
4,Cancelled,False,Set,0.0,NaN,HYDERABAD,TELANGANA,500008


In [7]:
# Merging the latitude/longitude information zipcode_df_cleaned['State/UT']:
latlng_info_df = pd.merge(add_latlng, 
                          coordinates_df, 
                          how='outer', on='State/UT')
latlng_info_df.head()

,Status,Status (Successful),Category,Qty,Amount,ship-city,State/UT,ship-postal-code,Latitude,Longitude
0,Delivered,True,Kurta,1.0,544.0,GUNTAKAL,ANDHRA PRADESH,515801.0,NaN,NaN
1,Delivered,True,Set,1.0,1233.0,VISAKHAPATNAM,ANDHRA PRADESH,530016.0,NaN,NaN
2,Cancelled,False,Kurta,0.0,NaN,GUWAHATI,ASSAM,781003.0,NaN,NaN
3,Delivered,True,Set,1.0,597.0,"GUWAHATI, KAMRUP (M)",ASSAM,781014.0,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,Andaman and Nicobar Islands,NaN,11.7401,92.6586


In [ ]:
# latlng_info_df["State/UT"].value_counts()

In [8]:
# Fix the capitalizations
latlng_info_df.loc[:,'State/UT'] = latlng_info_df['State/UT'].replace({
    'ANDAMAN AND NICOBAR ISLANDS': 'Andaman and Nicobar Islands',
    'ANDHRA PRADESH': 'Andhra Pradesh',
    'ARUNACHAL PRADESH': 'Arunachal Pradesh',
    'ASSAM': 'Assam',
    'BIHAR': 'Bihar',
    'CHHATTISGARH': 'Chhattisgarh',
    'BIHAR': 'Bihar',
    'DADRA AND NAGAR HAVELI AND DAMAN AND DIU': 'Dadra and Nagar Haveli and Daman and Diu',
    'DELHI': 'Delhi',
    'GOA': 'Goa',
    'GUJARAT': 'Gujarat',
    'HARYANA': 'Haryana',
    'JHARKHAND': 'Jharkhand',
    'KARNATAKA': 'Karnataka',
    'KERALA': 'Kerala',
    'LAKSHADWEEP': 'Lakshadweep',
    'MADHYA PRADESH': 'Madhya Pradesh',
    'MAHARASHTRA': 'Maharashtra',
    'MANIPUR': 'Manipur',
    'MEGHALYAYA': 'Meghalaya',
    'MIZORAM': 'Mizoram',
    'NAGALAND': 'Nagaland',
    'ODISHA': 'Odisha',
    'PUDUCHERRY': 'Puducherry',
    'PUNJAB': 'Punjab',
    'RAJASTHAN': 'Rajasthan',
    'SIKKIM': 'Sikkim',
    'TAMIL NADU': 'Tamil Nadu',
    'TELANGANA': 'Telangana',
    'TRIPURA': 'Tripura',
    'CHANDIGARH': 'Chandigarh',
    'JAMMU AND KASHMIR': 'Jammu and Kashmir',
    'LADAKH': 'Ladakh',
    'UTTAR PRADESH': 'Uttar Pradesh',
    'UTTARAKHAND': 'Uttarakhand',
    'WEST BENGAL': 'West Bengal'
    })

latlng_info_df = latlng_info_df.dropna(subset=['State/UT'])
latlng_info_df["State/UT"].value_counts()

State/UT
Maharashtra                                 9
Gujarat                                     5
Haryana                                     5
Karnataka                                   4
Telangana                                   4
Andhra Pradesh                              3
Uttar Pradesh                               3
Tamil Nadu                                  3
Odisha                                      3
Assam                                       3
Uttarakhand                                 2
Rajasthan                                   2
Puducherry                                  2
Madhya Pradesh                              2
West Bengal                                 2
Delhi                                       2
Jharkhand                                   2
Chhattisgarh                                2
Punjab                                      1
Andaman and Nicobar Islands                 1
Arunachal Pradesh                           1
Tripura                  

In [9]:
# Grouping the DataFrame by "Status"
group_delivered = latlng_info_df.groupby("Status")

# Count how many buildings were infested in each borough and create DataFrame
status_delivered = pd.DataFrame(group_delivered["State/UT"].value_counts())
status_delivered.head(10)

count
Status    State/UT          
Cancelled Gujarat          4
          Maharashtra      3
          Telangana        2
          Assam            1
          Delhi            1
          Puducherry       1
          Uttarakhand      1
Delivered Maharashtra      5
          Haryana          3
          Karnataka        3

In [12]:
latlng_info_df


,Status,Status (Successful),Category,Qty,Amount,ship-city,State/UT,ship-postal-code,Latitude,Longitude
0,Delivered,True,Kurta,1.0,544.00,GUNTAKAL,Andhra Pradesh,515801.0,NaN,NaN
1,Delivered,True,Set,1.0,1233.00,VISAKHAPATNAM,Andhra Pradesh,530016.0,NaN,NaN
2,Cancelled,False,Kurta,0.0,NaN,GUWAHATI,Assam,781003.0,NaN,NaN
3,Delivered,True,Set,1.0,597.00,"GUWAHATI, KAMRUP (M)",Assam,781014.0,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,Andaman and Nicobar Islands,NaN,11.7401,92.6586
...,...,...,...,...,...,...,...,...,...,...
70,Cancelled,False,Set,0.0,1105.36,DEHRADUN,Uttarakhand,248001.0,NaN,NaN
71,NaN,NaN,NaN,NaN,NaN,NaN,Uttar Pradesh,NaN,26.8467,80.9462
72,NaN,NaN,NaN,NaN,NaN,NaN,Uttarakhand,NaN,30.0668,79.0193
73,Delivered,True,Bottom,1.0,377.00,KOLKATA,West Bengal,700078.0,NaN,NaN


In [13]:
# Changing the column name to lat/lng
geo_latlng_info_df = latlng_info_df.rename(columns={'Latitude': 'Lat', 'Longitude': 'Lng'})
geo_latlng_info_df.head()

,Status,Status (Successful),Category,Qty,Amount,ship-city,State/UT,ship-postal-code,Lat,Lng
0,Delivered,True,Kurta,1.0,544.0,GUNTAKAL,Andhra Pradesh,515801.0,NaN,NaN
1,Delivered,True,Set,1.0,1233.0,VISAKHAPATNAM,Andhra Pradesh,530016.0,NaN,NaN
2,Cancelled,False,Kurta,0.0,NaN,GUWAHATI,Assam,781003.0,NaN,NaN
3,Delivered,True,Set,1.0,597.0,"GUWAHATI, KAMRUP (M)",Assam,781014.0,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,Andaman and Nicobar Islands,NaN,11.7401,92.6586


In [14]:
# Delivered
delivered_sts = geo_latlng_info_df.loc[geo_latlng_info_df['Status'] == 'Delivered']
delivered_sts.head()
# delivered_sts.to_csv("sample_Delivered_Status.csv", index=False)

,Status,Status (Successful),Category,Qty,Amount,ship-city,State/UT,ship-postal-code,Lat,Lng
0,Delivered,True,Kurta,1.0,544.0,GUNTAKAL,Andhra Pradesh,515801.0,NaN,NaN
1,Delivered,True,Set,1.0,1233.0,VISAKHAPATNAM,Andhra Pradesh,530016.0,NaN,NaN
3,Delivered,True,Set,1.0,597.0,"GUWAHATI, KAMRUP (M)",Assam,781014.0,NaN,NaN
9,Delivered,True,Kurta,1.0,299.0,BILASPUR,Chhattisgarh,495001.0,NaN,NaN
21,Delivered,True,Kurta,1.0,458.0,GURGAON,Haryana,122004.0,NaN,NaN


In [9]:
# Output 
# sales_product_info_df.to_csv("../output/cleaned_Product Sales Info.csv", index=False)

---

### Trying to create a map that displays a point for every shipping postal code (destination).

In [ ]:
%%capture --no-display

# Configure the map plot
# This is still the base from the homework, need to update the keys after
# figuring out the three status (total, delivered, unsuccessful)

shipped_state = geo_latlng_info_df["State/UT"]
map_plot_1 = geo_latlng_info_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 670,
    frame_height = 500,
    size = "Humidity",
    color="City",
    alpha=.75,
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_plot_1